# Carregando Dados 


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.functions import col, sum, avg


df_vendas = spark.read.csv("dbfs:/Volumes/workspace/default/tables/vendas.csv", header=True, inferSchema=True)
df_clientes = spark.read.csv("dbfs:/Volumes/workspace/default/tables/clientes.csv", header=True, inferSchema=True)
df_produtos = spark.read.csv("dbfs:/Volumes/workspace/default/tables/produtos.csv", header=True, inferSchema=True)

#Visualização dos dados

In [0]:
display(df_clientes)
display(df_produtos)
display(df_vendas)

#Verificando inconsistencia de dados

In [0]:
#Verificando se há valores nulos e dados inconsistentes em vendas

df_vendas.select(
    col("ProdutoID").isNull().cast("int").alias("ProdutoID_nulo"),
    col("Valor").isNull().cast("int").alias("Valor_nulo"),
    col("Quantidade").isNull().cast("int").alias("Quantidade_nulo")
).groupBy().sum().show()

df_vendas.filter((col("Valor") <= 0) | (col("Quantidade") <= 0)).display()

In [0]:
display(df_clientes)

In [0]:
#verificando se tem clientes com dados nulos
df_clientes.select(
    col("ClienteID").isNull().cast("int").alias("ClienteID_nulo"),
    col("Nome").isNull().cast("int").alias("Nome_nulo"),
    col("Idade").isNull().cast("int").alias("Idade_nulo"),
    col("Cidade").isNull().cast("int").alias("Cidade_nulo")
).groupBy().sum().display()

#Limpeza dos dados

In [0]:
#Removendo linhas que contem valores nulos
df_vendas = df_vendas.dropna(subset=["ProdutoID", "Valor", "Quantidade"]).filter((col("Valor") > 0) & (col("Quantidade") > 0))
df_clientes = df_clientes.dropna(subset=["ClienteID"]).filter(col("Idade") > 0)

In [0]:
#visualização final depois das linhas excluidas
display(df_vendas)
display(df_clientes)

#Adicionando coluna TotalVenda em Vendas

In [0]:
df_vendas = df_vendas.withColumn("TotalVenda", col("Valor") * col("Quantidade"))
display(df_vendas)

#junção de Dados - venda e clientes

In [0]:
display(df_vendas)

In [0]:
display(df_clientes)

In [0]:
df_vendas_clientes = df_vendas.join(df_clientes, df_vendas["ID"] == df_clientes["ClienteID"], how ="inner")
display(df_vendas_clientes)

#Junção de dados - Produtos

In [0]:
df_produtos_vendas_clientes = df_vendas_clientes.join(df_produtos, on="ProdutoID", how="inner")
display(df_produtos_vendas_clientes)

#Análise Simples - Total de vendas por produto

In [0]:
df_total_vendas_produto = df_produtos_vendas_clientes.groupBy("NomeProduto").agg(sum("TotalVenda").alias("TotalVendas"))
display(df_total_vendas_produto)

#Análise de Idade dos Clientes por Produto


In [0]:
df_media_idade_produto = df_produtos_vendas_clientes.groupBy("NomeProduto").agg(avg("Idade").alias("MediaIdadeClientes"))
display(df_media_idade_produto)

#Salvar os Dados Processados


In [0]:
df_media_idade_produto.write.mode("overwrite").parquet("dbfs:/Volumes/workspace/default/tables/vendas_clientes_produtos/")

#Consulta ao Conjunto de Dados Processados


In [0]:
df_vendas_clientes_produtos = spark.read.parquet("dbfs:/Volumes/workspace/default/tables/vendas_clientes_produtos/")
display(df_vendas_clientes_produtos)